In [107]:
import json 
import os 
import re
from transformers import AutoTokenizer

In [2]:
path = "./data/prompts/prism"
file = "prompts.json"

data = json.load(open(f"{path}/{file}"))

In [142]:
from stargate.vllm_inference_model import VLLMInferenceModel

model_config = {
    "model": "meta-llama/Meta-Llama-3.1-8B-Instruct",
    "download_dir": "/scr/jphilipp/stargate/pretrained_models/Meta-Llama-3.1-8B-Instruct",
    "dtype": "auto",
    "tensor_parallel_size": 1,
}

generation_config = {
    "max_new_tokens": 512,
    "num_return_sequences": 1,
    "temperature": 0.0
}

model = VLLMInferenceModel(
    **model_config
)


tokenizer = AutoTokenizer.from_pretrained(
    pretrained_model_name_or_path="meta-llama/Meta-Llama-3.1-8B-Instruct",
    cache_dir="/scr/jphilipp/stargate/pretrained_models/Meta-Llama-3.1-8B-Instruct",
)

In [12]:
prompts = [
    [   
        {"role": "user", "content": "being a first year grad student can be confusing"},
    ]
]

formatted_prompts = [
    tokenizer.apply_chat_template(prompt, tokenize=False) for prompt in prompts
]

print(formatted_prompts[0])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

being a first year grad student can be confusing<|eot_id|>


In [22]:
response = model.batch_prompt(prompts=formatted_prompts, **generation_config)

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.10s/it, est. speed input: 13.22 toks/s, output: 82.56 toks/s]


In [120]:
import re 
def extract_response(response, key):
    escaped_key = re.escape(key)
    pattern = rf"<{escaped_key}>(.*?)</{escaped_key}>"
    match = re.search(pattern, response, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return f"{key.capitalize()} tag not found in the response."

In [147]:
def get_context_prompt(opening_prompt):
    return f"""\
Given the query below, think about an interesting context—i.e., a situation that might influence how one should respond to the query. Be creative and non-generic, it must be a diverse and unexpected context that can drive the response to the query.

<query>{opening_prompt}</query>

Format your response as follows:
<reasoning>Your step-by-step reasoning thinking about what a potentially interesting context could be in here. No more than 50 words.</reasoning>
<context>Your generated context here</context>
<additional_comments>Any additional thoughts here</additional_comments?
"""

def get_response_prompt(opening_prompt, context):
    return f"""\
Given the query and context below, return a great response the query.

<query>{opening_prompt}</query>
<context>{context}</context>

Format your response as follows:
<response>Your response here</response>
"""

In [148]:
input_data = {"opening_prompt": "what are some names for a new tiktok account im making"}

In [149]:
prompts = []
prompts.append([{"role": "user", "content": get_context_prompt(**input_data)}])

formatted_prompts = [
    tokenizer.apply_chat_template(prompt, tokenize=False) for prompt in prompts
]

print(formatted_prompts[0])

response = model.batch_prompt(prompts=formatted_prompts, **generation_config)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the query below, think about an interesting context—i.e., a situation that might influence how one should respond to the query. Be creative and non-generic, it must be a diverse and unexpected context that can drive the response to the query.

<query>being a first year grad student can be confusing</query>

Format your response as follows:
<reasoning>Your step-by-step reasoning thinking about what a potentially interesting context could be in here. No more than 50 words.</reasoning>
<context>Your generated context here</context>
<additional_comments>Any additional thoughts here</additional_comments?<|eot_id|>


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.19s/it, est. speed input: 69.77 toks/s, output: 82.08 toks/s]


In [154]:
input_data["context"] = extract_response(response[0], key="context")
prompts = []
prompts.append([{"role": "user", "content": get_response_prompt(**input_data)}])

formatted_prompts = [
    tokenizer.apply_chat_template(prompt, tokenize=False) for prompt in prompts
]

print(formatted_prompts[0])

response = model.batch_prompt(prompts=formatted_prompts, **generation_config)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Given the query and context below, return a great response the query.

<query>being a first year grad student can be confusing</query>
<context>A 25-year-old cellist, who has spent their entire life performing with orchestras and teaching privately, is now enrolled in a Master's program in musicology at a prestigious university. They're finding it challenging to balance their academic coursework with their performance schedule and the pressure to produce research that meets the expectations of their professors.</context>

Format your response as follows:
<response>Your response here</response><|eot_id|>


Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.32s/it, est. speed input: 33.38 toks/s, output: 82.29 toks/s]


In [156]:
print(extract_response(response[0], key="response"))

It's completely normal to feel overwhelmed as a first-year grad student, especially when you're transitioning from a performance-based career to an academic one. As a cellist, you're likely used to a structured routine and a clear set of goals, such as perfecting a piece or preparing for a performance. In contrast, graduate school requires a different kind of discipline and time management.

Firstly, take a deep breath and acknowledge that it's okay to not have it all figured out right away. It's a process to adjust to the academic environment, and it's normal to feel like you're navigating uncharted territory. 

To better manage your time, consider creating a schedule that balances your coursework, performance commitments, and research. Be realistic about how much time you can dedicate to each activity, and prioritize your tasks accordingly. Make sure to leave some buffer time for unexpected events or last-minute requests from your professors.

It might also be helpful to connect with